In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import Geohash as gh
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    client = MongoClient()
    db = client.fit5148_assignment_db
    isfire = db.isfire
    cli = db.cli
    
    datas = []
   
    for record in iter:
        temp = json.loads(record[1])
        temp['geo'] = gh.encode(temp['latitude'],temp['longitude'],precision=5)   
        datas.append(temp)  
        
        if temp['sender_id'] == 'producer1':
            try:
                cli.insert_one(temp)
            except Exception as ex:
                print("Exception Occured. Message: {0}".format(str(ex)))
    
    print('no.of datas' + str(len(datas)))
    
    if len(datas)== 3:
        pro1 = []
        pro2 = []
        for x in datas:
            if x['sender_id'] == 'producer1':
                pro1.append(x)
            else:
                pro2.append(x)
        
        for i in range(len(pro1)):
            if pro1[i]['geo'] == pro2[0]['geo']:
                try:
                    isfire.insert_one(pro2[0])
                    break
                except Exception as ex:
                    print("Exception Occured. Message: {0}".format(str(ex)))
           
    if len(datas)== 4:
        pro1 = []
        pro2 = []
        
        for x in datas:
            if x['sender_id'] == 'producer1':
                pro1.append(x)
            else:
                pro2.append(x)
        

        if len(pro2)== 2:
            if pro2[0]['geo'] == pro2[1]['geo']:
                avr_surface = ((int)(pro2[0]['surface_temperature_celcius']) + (int)(pro2[1]['surface_temperature_celcius']))/2
                avr_conf = ((int)(pro2[0]['confidence']) + (int)(pro2[1]['confidence']))/2
                
                pro2[0]['surface_temperature_celcius'] = avr_surface
                pro2[0]['confidence'] = avr_conf
                
                for y in pro1:
                    if pro2[0]['geo'] == y['geo']:
                        try:
                            isfire.insert_one(pro2[0])
                            break
                        except Exception as ex:
                            print("Exception Occured. Message: {0}".format(str(ex)))
            
            else:
                counter = 0
                
                if counter < 2:
                    for entry1 in pro1:
                        for entry2 in pro2:
                            if entry1['geo'] == entry2['geo']:
                                try:
                                    isfire.insert_one(entry2)
                                    counter = counter + 1
                                except Exception as ex:
                                    print("Exception Occured. Message: {0}".format(str(ex)))

                        
                    
    
    client.close()

n_secs = 10
topic = "fire"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week12-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)